<a href="https://colab.research.google.com/github/KVincent007/Predicting-age-from-the-transcriptome-of-human-dermal-fibroblasts/blob/master/2018Fleischer_replicate_cleancode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transcriptomic Age Prediction (replication attempt of 2018 model by Jason)

import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error, r2_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# --- Load Data ---
df = pd.read_csv('/content/merged_TPM_with_metadata_filtered (1).csv', index_col=0)
X_full = df.drop(columns=['Age', 'sex', 'disease'])
y = df['Age']
y_cls = pd.cut(y, bins=[0, 21, 41, 61, 81, 100], labels=['0-20', '21-40', '41-60', '61-80', '81-100'])

# --- CV Setup ---
rkf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=42)

# --- Storage ---
results = {
    "ElasticNet": [], "SVR": [], "Linear": [],
    "LDA_mae": [], "LDA_r2": []
}
true_ages = []
pred_ages_lda = []

# --- Age bin midpoints ---
age_bin_to_mid = {'0-20': 10, '21-40': 30, '41-60': 50, '61-80': 70, '81-100': 90}

# --- Cross-Validation Loop ---
for fold, (train_idx, test_idx) in enumerate(rkf.split(X_full, y_cls), 1):
    X_train_raw, X_test_raw = X_full.iloc[train_idx], X_full.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    y_cls_train = y_cls.iloc[train_idx]
    y_test_cls = y_cls.iloc[test_idx]

    # --- Fold-Specific Gene Filtering (original) ---
    #gene_mask = []
    #for gene in X_train_raw.columns:
    #    gene_vals = X_train_raw[gene]
    #    if gene_vals.max() > 5 and (gene_vals.max() / gene_vals.replace(0, np.nan).min()) >= 5:
    #        gene_mask.append(gene)
    #X_train = X_train_raw[gene_mask].copy()
    #X_test = X_test_raw[gene_mask].copy()

    gene_mask = []
    for gene in X_train_raw.columns:
        gene_vals = X_train_raw[gene]
        if gene_vals.max() > 5:
            nonzero = gene_vals[gene_vals > 0]
            if not nonzero.empty and gene_vals.max() / nonzero.min() >= 5:
                gene_mask.append(gene)

    if not gene_mask:
        raise ValueError(f"No genes passed filtering in fold {fold}")


    # To ensure X-train and y-train always have the same number of rows
    X_train = X_train_raw[gene_mask].copy()
    X_test  = X_test_raw[ gene_mask].copy()

    # Now the shapes will match exactly:
    #   X_train.shape[1] == X_test.shape[1]  AND
    #   X_train.shape[0] == len(y_train)

    # --- Log-transform + Standardize ---
    X_train = np.log2(X_train + 1)
    X_test = np.log2(X_test + 1)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # --- ElasticNet ---
    enet_pub = ElasticNet(alpha=0.1, l1_ratio=0.0, random_state=42, max_iter=10000, tol=0.01)
    enet_pub.fit(X_train_scaled, y_train)
    enet_pred = enet_pub.predict(X_test_scaled)
    results['ElasticNet'].append((mean_absolute_error(y_test, enet_pred), r2_score(y_test, enet_pred)))

    # --- SVR ---
    svr_pub = SVR(kernel='poly', degree=2, C=10, epsilon=0.05, gamma=0.0002)
    svr_pub.fit(X_train_scaled, y_train)
    svr_pred = svr_pub.predict(X_test_scaled)
    results['SVR'].append((mean_absolute_error(y_test, svr_pred), r2_score(y_test, svr_pred)))

    # --- Linear Regression ---
    lr = LinearRegression()
    lr.fit(X_train_scaled, y_train)
    lr_pred = lr.predict(X_test_scaled)
    results['Linear'].append((mean_absolute_error(y_test, lr_pred), r2_score(y_test, lr_pred)))

    # --- LDA (classification) ---
    lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
    lda.fit(X_train_scaled, y_cls_train)
    y_pred_cls = lda.predict(X_test_scaled)
    y_pred_num = [age_bin_to_mid[str(c)] for c in y_pred_cls]
    true_ages.extend(y.iloc[test_idx])
    pred_ages_lda.extend(y_pred_num)

# --- Evaluation ---
def summarize_model(name, metrics):
    mae = np.mean([m[0] for m in metrics])
    r2 = np.mean([m[1] for m in metrics])
    print(f"{name} → MAE: {mae:.2f} | R²: {r2:.2f}")

print("\n📊 Repeated CV Summary:")
summarize_model("ElasticNet", results["ElasticNet"])
summarize_model("SVR", results["SVR"])
summarize_model("Linear", results["Linear"])

# LDA evaluation
lda_mae = mean_absolute_error(true_ages, pred_ages_lda)
lda_r2 = r2_score(true_ages, pred_ages_lda)
print(f"LDA → MAE: {lda_mae:.2f} | R²: {lda_r2:.2f}")



📊 Repeated CV Summary:
ElasticNet → MAE: 12.78 | R²: 0.67
SVR → MAE: 18.33 | R²: 0.38
Linear → MAE: 11.85 | R²: 0.71
LDA → MAE: 9.28 | R²: 0.73


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore")

# --- Load Data ---
df = pd.read_csv('/content/merged_TPM_with_metadata_filtered (1).csv', index_col=0)
X_full = df.drop(columns=['Age', 'sex', 'disease'])
y = df['Age']
y_cls = pd.cut(y, bins=[0, 21, 41, 61, 81, 100],
               labels=['0-20', '21-40', '41-60', '61-80', '81-100'],
               include_lowest=True)

# --- CV Setup ---
outer_cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=42)

# parameter grids for inner search
enet_grid = {
    'alpha': [0.01, 0.1, 1.0],
    'l1_ratio': [0.0, 0.5, 1.0]
}
svr_grid = {
    'kernel': ['poly', 'rbf'],
    'degree': [2, 3],             # only for poly
    'C': [1, 10],
    'epsilon': [0.05, 0.1],
    'gamma': ['scale', 0.0002]
}

results = {'ElasticNet': [], 'SVR': [], 'Linear': [], 'LDA': []}
true_ages, pred_ages_lda = [], []
age_bin_to_mid = {'0-20':10,'21-40':30,'41-60':50,'61-80':70,'81-100':90}

for train_idx, test_idx in outer_cv.split(X_full, y_cls):
    X_train_raw, X_test_raw = X_full.iloc[train_idx], X_full.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    y_cls_train = y_cls.iloc[train_idx]

    # your fold-specific gene filtering
    mask = []
    for g in X_train_raw.columns:
        vals = X_train_raw[g]
        if vals.max()>5 and (vals.max()/vals.replace(0,np.nan).min())>=5:
            mask.append(g)
    X_train = np.log2(X_train_raw[mask] + 1)
    X_test  = np.log2(X_test_raw[mask]  + 1)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s  = scaler.transform(X_test)

    # --- ElasticNet with inner GridSearchCV ---
    inner_cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=42)
    enet = GridSearchCV(ElasticNet(max_iter=5000), enet_grid,
                        cv=inner_cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    enet.fit(X_train_s, y_train)
    pred = enet.best_estimator_.predict(X_test_s)
    results['ElasticNet'].append((mean_absolute_error(y_test, pred),
                                  r2_score(y_test, pred)))

    # --- SVR with inner GridSearchCV ---
    svr = GridSearchCV(SVR(), svr_grid,
                       cv=inner_cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    svr.fit(X_train_s, y_train)
    pred = svr.best_estimator_.predict(X_test_s)
    results['SVR'].append((mean_absolute_error(y_test, pred),
                           r2_score(y_test, pred)))

    # --- Linear Regression (no tuning) ---
    lr = LinearRegression()
    lr.fit(X_train_s, y_train)
    pred = lr.predict(X_test_s)
    results['Linear'].append((mean_absolute_error(y_test, pred),
                              r2_score(y_test, pred)))

    # --- LDA (no tuning) ---
    lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
    lda.fit(X_train_s, y_cls_train)
    cls_pred = lda.predict(X_test_s)
    true_ages.extend(y_test)
    pred_ages_lda.extend([age_bin_to_mid[str(c)] for c in cls_pred])

# --- Summarize ---
def summarize(name, vals):
    mae = np.mean([v[0] for v in vals]); r2 = np.mean([v[1] for v in vals])
    print(f"{name} → MAE: {mae:.2f} | R²: {r2:.2f}")

print("\nRegression Models:")
summarize("ElasticNet", results['ElasticNet'])
summarize("SVR", results['SVR'])
summarize("LinearRegression", results['Linear'])

from sklearn.metrics import mean_absolute_error, r2_score
print("\nLDA →", end=" ")
print(f"MAE: {mean_absolute_error(true_ages, pred_ages_lda):.2f} | "
      f"R²: {r2_score(true_ages, pred_ages_lda):.2f}")



Regression Models:
ElasticNet → MAE: 12.09 | R²: 0.69
SVR → MAE: 17.84 | R²: 0.40
LinearRegression → MAE: 11.85 | R²: 0.71

LDA → MAE: 9.28 | R²: 0.73
